# Importing and preparing sbb data

## Libraries and setting

In [1]:
import os
import re
import time
import fnmatch
import numpy as np
import pandas as pd
import json

import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())

c:\Users\Fabian\sbe\SBBAnalytics


## Importing data

In [2]:
df = pd.read_json('passagierfrequenz.json', encoding='utf-8')
df.head()

,code_codice,uic,bahnhof_gare_stazione,kt_ct_cantone,isb_gi,jahr_annee_anno,dtv_tjm_tgm,dwv_tmjo_tfm,dnwv_tmjno_tmgnl,evu_ef_itf,bemerkungen,remarques,note,remarks,geopos,lod
0,AAT,8503124,Aathal,ZH,SBB,2018,740,800,610,SBB,None,None,None,None,"{'lon': 8.765625135347072, 'lat': 47.335959133...",http://lod.opentransportdata.swiss/didok/didok85
1,AB,8507472,Ausserberg,VS,BLS,2018,180,160,210,BLS,None,None,None,None,"{'lon': 7.8441670280993065, 'lat': 46.31234824...",http://lod.opentransportdata.swiss/didok/didok85
2,ABE,8504404,Aarberg,BE,SBB,2022,520,580,400,BLS,None,None,None,None,"{'lon': 7.278692565659158, 'lat': 47.044541277...",http://lod.opentransportdata.swiss/didok/didok85
3,ABE,8504404,Aarberg,BE,SBB,2023,580,620,460,BLS,None,None,None,None,"{'lon': 7.278692565659158, 'lat': 47.044541277...",http://lod.opentransportdata.swiss/didok/didok85
4,ABO,8502000,Aarburg-Oftringen,AG,SBB,2022,2500,2900,1500,SBB,None,None,None,None,"{'lon': 7.908222606719322, 'lat': 47.320268469...",http://lod.opentransportdata.swiss/didok/didok85


## Get data types

In [3]:
df.dtypes

code_codice              object
uic                       int64
bahnhof_gare_stazione    object
kt_ct_cantone            object
isb_gi                   object
jahr_annee_anno           int64
dtv_tjm_tgm               int64
dwv_tmjo_tfm              int64
dnwv_tmjno_tmgnl          int64
evu_ef_itf               object
bemerkungen              object
remarques                object
note                     object
remarks                  object
geopos                   object
lod                      object
dtype: object

## Delete columns

In [4]:
df.drop(columns=["bemerkungen", "remarques", "note", "remarks", "lod"], inplace=True)
df.head()

,code_codice,uic,bahnhof_gare_stazione,kt_ct_cantone,isb_gi,jahr_annee_anno,dtv_tjm_tgm,dwv_tmjo_tfm,dnwv_tmjno_tmgnl,evu_ef_itf,geopos
0,AAT,8503124,Aathal,ZH,SBB,2018,740,800,610,SBB,"{'lon': 8.765625135347072, 'lat': 47.335959133..."
1,AB,8507472,Ausserberg,VS,BLS,2018,180,160,210,BLS,"{'lon': 7.8441670280993065, 'lat': 46.31234824..."
2,ABE,8504404,Aarberg,BE,SBB,2022,520,580,400,BLS,"{'lon': 7.278692565659158, 'lat': 47.044541277..."
3,ABE,8504404,Aarberg,BE,SBB,2023,580,620,460,BLS,"{'lon': 7.278692565659158, 'lat': 47.044541277..."
4,ABO,8502000,Aarburg-Oftringen,AG,SBB,2022,2500,2900,1500,SBB,"{'lon': 7.908222606719322, 'lat': 47.320268469..."


## Count, identify and remove missing values

In [5]:
print('Count missing values per variable')
print(pd.isna(df).sum(), '\n')

print('Identify rows with missing values')

print(df.loc[df.isna().any(axis=1)][["code_codice", "uic"]], '\n')

df1 = df.dropna()
df1.head()

Count missing values per variable
code_codice              0
uic                      0
bahnhof_gare_stazione    0
kt_ct_cantone            0
isb_gi                   0
jahr_annee_anno          0
dtv_tjm_tgm              0
dwv_tmjo_tfm             0
dnwv_tmjno_tmgnl         0
evu_ef_itf               0
geopos                   9
dtype: int64 

Identify rows with missing values
     code_codice      uic
985           JE  8503421
1038         LOT  8503420
1039         LOT  8503420
1311         TIR  8509369
1695          JE  8503421
1765         LOT  8503420
2012         TIR  8509369
2013         TIR  8509369
3111          JE  8503421 



,code_codice,uic,bahnhof_gare_stazione,kt_ct_cantone,isb_gi,jahr_annee_anno,dtv_tjm_tgm,dwv_tmjo_tfm,dnwv_tmjno_tmgnl,evu_ef_itf,geopos
0,AAT,8503124,Aathal,ZH,SBB,2018,740,800,610,SBB,"{'lon': 8.765625135347072, 'lat': 47.335959133..."
1,AB,8507472,Ausserberg,VS,BLS,2018,180,160,210,BLS,"{'lon': 7.8441670280993065, 'lat': 46.31234824..."
2,ABE,8504404,Aarberg,BE,SBB,2022,520,580,400,BLS,"{'lon': 7.278692565659158, 'lat': 47.044541277..."
3,ABE,8504404,Aarberg,BE,SBB,2023,580,620,460,BLS,"{'lon': 7.278692565659158, 'lat': 47.044541277..."
4,ABO,8502000,Aarburg-Oftringen,AG,SBB,2022,2500,2900,1500,SBB,"{'lon': 7.908222606719322, 'lat': 47.320268469..."


## Count, identify & remove duplicated values

In [6]:
# Convert dictionary columns to strings
df1['geopos'] = df1['geopos'].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)

print('Sum of missing values:', df1.duplicated().sum(), '\n')

print('Duplicated values')
print(df1.loc[df1.duplicated(keep='last')])

# Drop all duplicate values
df2 = df1.drop_duplicates()

Sum of missing values: 1 

Duplicated values
    code_codice      uic bahnhof_gare_stazione kt_ct_cantone isb_gi  \
551         SMG  8506314     St. Margrethen SG            SG    SBB   

     jahr_annee_anno  dtv_tjm_tgm  dwv_tmjo_tfm  dnwv_tmjno_tmgnl  \
551             2023         4200          4600              3400   

           evu_ef_itf                                             geopos  
551  SBB, SOB, Thurbo  {"lon": 9.638261920244418, "lat": 47.453215993...  


## Change the column names

In [7]:
df2.rename(columns={'code_codice': 'Code'}, inplace=True)

df2.rename(columns={'uic': 'UIC'}, inplace=True)

df2.rename(columns={'isb_gi': 'ISB_GI'}, inplace=True)

df2.rename(columns={'bahnhof_gare_stazione': 'Bahnhof'}, inplace=True)

df2.rename(columns={'kt_ct_cantone': 'Kanton'}, inplace=True)

df2.rename(columns={'jahr_annee_anno': 'Jahr'}, inplace=True)

df2.rename(columns={'dtv_tjm_tgm': 'DTV'}, inplace=True)

df2.rename(columns={'dwv_tmjo_tfm': 'DWV'}, inplace=True)

df2.rename(columns={'dnwv_tmjno_tmgnl': 'DNWV'}, inplace=True)

df2.rename(columns={'evu_ef_itf': 'EVU'}, inplace=True)

df2.rename(columns={'geopos': 'Geoposition'}, inplace=True)

df2.head()

,Code,UIC,Bahnhof,Kanton,ISB_GI,Jahr,DTV,DWV,DNWV,EVU,Geoposition
0,AAT,8503124,Aathal,ZH,SBB,2018,740,800,610,SBB,"{""lon"": 8.765625135347072, ""lat"": 47.335959133..."
1,AB,8507472,Ausserberg,VS,BLS,2018,180,160,210,BLS,"{""lon"": 7.8441670280993065, ""lat"": 46.31234824..."
2,ABE,8504404,Aarberg,BE,SBB,2022,520,580,400,BLS,"{""lon"": 7.278692565659158, ""lat"": 47.044541277..."
3,ABE,8504404,Aarberg,BE,SBB,2023,580,620,460,BLS,"{""lon"": 7.278692565659158, ""lat"": 47.044541277..."
4,ABO,8502000,Aarburg-Oftringen,AG,SBB,2022,2500,2900,1500,SBB,"{""lon"": 7.908222606719322, ""lat"": 47.320268469..."


## Creating columns lon and lat

In [8]:
# Extract 'lon' and 'lat' from 'Geoposition' column
df2[['lon', 'lat']] = df2['Geoposition'].apply(lambda x: pd.Series(json.loads(x)))

# Drop the original 'Geoposition' column
df2.drop(columns=['Geoposition'], inplace=True)

df2.head()

,Code,UIC,Bahnhof,Kanton,ISB_GI,Jahr,DTV,DWV,DNWV,EVU,lon,lat
0,AAT,8503124,Aathal,ZH,SBB,2018,740,800,610,SBB,8.765625,47.335959
1,AB,8507472,Ausserberg,VS,BLS,2018,180,160,210,BLS,7.844167,46.312348
2,ABE,8504404,Aarberg,BE,SBB,2022,520,580,400,BLS,7.278693,47.044541
3,ABE,8504404,Aarberg,BE,SBB,2023,580,620,460,BLS,7.278693,47.044541
4,ABO,8502000,Aarburg-Oftringen,AG,SBB,2022,2500,2900,1500,SBB,7.908223,47.320268


## Save data to file

In [9]:
df2.to_csv('data_prepared.csv', 
            sep=",", 
            encoding='utf-8',
            index=False)

In [10]:
!pip install pandas